### Updating The FX_derivatives_all database 
Update with the TJ06 and TX06sap reports.

In [30]:

import pandas as pd
import csv


fxfile = []
df = pd.read_csv(r'\\or1isi1vl3921nfs\sapnfsc\PRD003\0256\TX06_Extract09212020.txt',
                 delimiter = '|')


    
    


In [ ]:
#let's rename our column headers without spaces so that we can then separate columns (currently 1 column) into many.

In [31]:
df

------------------------------------------------------------------------------------------------------------------------------
NaN                                                 CoCd Trans. Partner Pmnt Date   Rate          Purch. Crc  Purch.Amnt     Sale Curre  Sale Amount    Contract   PTyp TTyp                                                NaN                                                                             
-----------------------------------------------... NaN   NaN    NaN     NaN        NaN            NaN        NaN             NaN        NaN             NaN        NaN  NaN                                                 NaN                                                                             
NaN                                                 ADUS 2279   2       10/15/2020  4.536576000   USD         4,849,472.38   RON         22,000,000.00  05/06/2020 FXF  102                                                 NaN                                                                             
                                                    ADIR 8004   2101    11/19/2020 107.388900000  USD         44,697,356.99  JPY         4,800,000,000  05/11/2020 FXF  102                                                 NaN                                                                             
                                                         8006   2081    11/19/2020  25.376190000  CZK         32,000,000.00  USD         1,261,024.61   05/11/2020 FXF  102                                                 NaN                                                                             
...                                                                                                                                                                                                                         ...                                                                             
                                                         7997   12600   10/16/2020  9.823572700   SEK         12,000,000.00  USD         1,221,551.50   05/06/2020 FXF  102                                                 NaN                                                                             
                                                         8105   2101    11/18/2020  0.644818100   NZD         4,000,000.00   USD         2,579,272.40   06/16/2020 FXF  102                                                 NaN                                                                             
                                                         8185   2101    10/08/2020  0.904118400   USD         3,318,149.48   CHF         3,000,000.00   08/05/2020 FXF  102                                                 NaN                                                                             
                                                         8205   2       12/17/2020  1.179288000   EUR         35,000,000.00  USD         41,275,080.00  08/11/2020 FXF  102                                                 NaN                                                                             
-----------------------------------------------... NaN   NaN    NaN     NaN        NaN            NaN        NaN             NaN        NaN             NaN        NaN  NaN                                                 NaN                                                                             

[66 rows x 1 columns]

In [34]:
git commit -a -m 'import fxcsv'

SyntaxError: invalid syntax (<ipython-input-34-9acc4f743069>, line 1)